# Statistical Analysis

Statistics is the science of collecting, analyzing, presenting, and interpreting data.
## Descriptive Analysis

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Doctorate Recipients by Demography--- Doctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df = pd.read_csv('../../Users/gitData/SED_export_table_2020-11-12T19_44_35.595Z.csv', skiprows=[0], header=[1])

# Removing 'Total values' --- for calculating statistical purposes
df1 = df['Detailed Fields']!='Total for selected values'
df = df[df1]
df2 = df['Race and Ethnicity']!='Total for selected values'
df = df[df2]
df3 = df['Citizenship']!='Total for selected values'
df = df[df3]
df4 = df['Sex']!='Total for selected values'
df = df[df4]
df.head()

In [ ]:
# checking the data types of the columns in the dataset
df.info()

In [ ]:
# checking for duplicate values in the data
df.duplicated().sum()

In [ ]:
# let's check for missing values in the data
round(df.isnull().sum() / df.isnull().count() * 100, 2)

In [ ]:
df = df.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Race and Ethnicity',
       'Citizenship', 'Sex'])
df.head()

In [ ]:
df = df.stack().to_frame()
df = df.reset_index()
df.head()

In [ ]:
df.columns = ['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Race and Ethnicity',
       'Citizenship', 'Sex', 'Year', 'Number']
df.head()
#df.columns
df.shape

In [ ]:
df3 = df['Number']!='-'
df = df[df3]
df.dtypes

In [ ]:
df['Number']=df['Number'].str.replace(',','')  # Removing ',' from numbers to change object to int

df.info()

In [ ]:
df["Number"] = df["Number"].astype(str).astype(int)
df.dtypes
#df["Number"] = pd.to_numeric(df["Number"])

### Common data manipulation tasks

1. Selecting/Indexing
2. Filtering
3. Sorting
4. Mutating/Conditionally adding documents
5. Groupby/Summarize

In [ ]:
# selecting/Indexing 
df1 =  df.iloc[1:10, :]

df1.head(15)

In [ ]:
# Filtering
df2 = df[df["Broad Fields"] == "Life sciences"]

df2.head()

In [ ]:
# Sorting
df1.sort_values(["Number"],  ascending=False)

In [ ]:
df[['Year', 'Number']].groupby(["Year"]).sum()

In [ ]:
df[['S&E Fields',"Year", 'Number']].groupby(['S&E Fields',"Year"]).mean()

In [ ]:
df[['S&E Fields',"Year", 'Number']].groupby(['S&E Fields',"Year"]).sum()

In [ ]:
df.head()

In [ ]:
df.groupby('Broad Fields').sum()

In [ ]:
df.groupby('Broad Fields')['Number'].sum()